In [170]:
import serial
import struct
import time
import numpy as np
from serial.tools.list_ports import comports

def read_number(msg):
    while True:
        try:
            #return 2;
            return int(input(msg))
        except: print("ERROR: Not a number")

def read_port(msg, available_ports):
    while True:
        try:
            port = input(msg)
            #index = input(msg)
            #port = "COM8";
            return serial.Serial(port, 9600)
        except: print(f"ERROR: Wrong port connection ({available_ports[index-1]})")
            
def print_until_keyword(keyword, arduino):
    while True: 
        msg = arduino.readline().decode()
        if msg[:-2] == keyword: break
        else: print(f'({arduino.port}):',msg, end='')
            
def read_matrix(device, dimms):
    result = np.zeros((1,dimms)).reshape(-1)
    for i in range(dimms):
        device.read()
        result[i] = struct.unpack('f', device.read(4))[0]
    
    return result.reshape(dimms)


In [171]:
def getDevices():
    # num_devices = read_number("Number of devices: ")
    print("Hardcoded to 1 device.")
    num_devices = 1

    available_ports = comports()
    print("Available ports:")
    for i,available_port in enumerate(available_ports): print(f"{available_port}")

    devices = [read_port(f"Port device_{i+1}: ", available_ports) for i in range(num_devices)]
    return devices


In [172]:
# Send the blank model to all the devices
def receive_model_info(device):
    device.reset_input_buffer()
    device.write(b's') # Python --> ACK --> Arduino
    print_until_keyword('start', device) # CLEAN SERIAL
    
    bytesToRead = device.read(1).decode()
    time.sleep(1)
    if bytesToRead == 'i':
        [num_layers] = struct.unpack('i', device.read(4))
        dimms = []
        for i in range(num_layers):
            [rows, cols] = struct.unpack('ii', device.read(8))
            dimms.append((1,cols)) # bias
            dimms.append((rows,cols)) # matrix weigths
    return num_layers, dimms


In [173]:
def initialize_device_weights(device, layer, bias_dimm, w_dimm):
    bias = np.zeros(bias_dimm)
    weights = np.random.randn(w_dimm[0], w_dimm[1]) * np.sqrt(6.0 / (w_dimm[0] + w_dimm[1]))
    print(f"Sending weights for layer {layer}")
    for b in bias.reshape(-1):
        data = device.read()
        device.write(struct.pack('f', b))

    for w in weights.reshape(-1):
        data = device.read()
        device.write(struct.pack('f', w))
    

In [174]:
def send_sample(device, X, y=None):
    for s in X.reshape(-1):
        data = device.read()
        device.write(struct.pack('f', s))
    if y is not None:
        for t in y.reshape(-1):
            data = device.read()
            device.write(struct.pack('f', t))

def get_tick():
    return round(time.time() * 1000)

def train(device, X, y):
    error = 0.0
    start = get_tick()
    for i in range(1000):
        device.write(b"t")
        send_sample(device, X[i], y[i])
        n_error = device.read(4)
        loss = struct.unpack('f', n_error)[0]
        error += loss
    end = get_tick()
    return end-start, error/1000.0

def predict(device, X):
    start = get_tick()
    device.write(b"p")
    send_sample(device, X)
    # read last layer size output
    output = read_matrix(device, dimms[-1][1])
    return get_tick() - start, output


In [175]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder

X,y = load_digits(return_X_y=True)
onehot_encoder = OneHotEncoder(sparse=False)
y_train_one_hot = onehot_encoder.fit_transform(y.reshape(-1,1))

In [176]:
devices = getDevices()
num_layers, dimms = receive_model_info(devices[0])
    
for device in devices:
    for i in range(0,len(dimms),2):
        initialize_device_weights(device,i//2,dimms[i], dimms[i+1])


Hardcoded to 1 device.
Available ports:
/dev/cu.wlan-debug - n/a
/dev/cu.Bluetooth-Incoming-Port - n/a
/dev/cu.usbmodem101 - Envie M7
Port device_1: /dev/cu.usbmodem101
(/dev/cu.usbmodem101): g for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101): Waiting for new model...
(/dev/cu.usbmodem101)

Sending weights for layer 0
Sending weights for layer 1
Sending weights for layer 2


In [194]:
for i in range(5):
    dt, loss = train(devices[0], X, y_train_one_hot)
    print(dt, loss)

9008 0.07303109507169575
8902 0.06730819769110531
9092 0.06012059526145458
8961 0.056933463068678976
8867 0.0537441280935891


In [196]:
import matplotlib.pyplot as plt
acc = 0.0
for i in range(797):
    dt, res = predict(devices[0], X[1000+i])
    acc += 1 if y[1000+i] == np.argmax(res) else 0
acc / 797.0

0.8971141781681304